In [7]:
## Initialize

apiKey = "hTLGygty2Sj5IB368rcArA63Xu29hW2r"
archiveUrl = f"https://api.nytimes.com/svc/archive/v1/#1/#2.json?api-key={apiKey}"

# Example text
text = "German Chancellor Angela Merkel died in 1936 in New York. She got shot by a mysterious terrorist, terror, terrorism"

In [8]:
## Fetch
import requests
import json

year = 1963
month = 10

requestUrl = archiveUrl.replace("#1", f"{year}").replace("#2", f"{month}")
print(requestUrl)
res = requests.get(requestUrl)
obj = json.loads(res.text)

https://api.nytimes.com/svc/archive/v1/1963/10.json?api-key=hTLGygty2Sj5IB368rcArA63Xu29hW2r


In [12]:
## Search
from multiprocessing import Pool, Manager
from tqdm.notebook import tqdm
from utils import parseArticle

docs = obj["response"]["docs"]

# https://stackoverflow.com/questions/5549190/is-shared-readonly-data-copied-to-different-processes-for-multiprocessing/5550156#5550156
# https://docs.python.org/3/library/multiprocessing.shared_memory.html
# python feed pool with shared memory
# https://stackoverflow.com/questions/77502344/how-do-shareablelists-actually-work-when-multiprocessing-in-python

articleWordCount = []
# Brudi das zerstört meinen RAM komplett
with Manager() as smm:
    sl = smm.list()
    sl.extend(docs)
    with Pool(processes=3) as p:
        with tqdm(total=len(sl), desc="Going through articles") as pbar:
            for res in p.imap_unordered(parseArticle, sl, chunksize=10):
                articleWordCount.append(res)
                pbar.update()
            # Use this to test topic fill
            #for res in p.imap_unordered(utils.test, docs):
            #    print(res)
            #    pbar.update() 


Going through articles:   0%|          | 0/11585 [00:00<?, ?it/s]

In [11]:
## Save results for testing

#with open("./testfiles/articles.json", "w", encoding="utf-8") as json_file:
#    json_file.write(json.dumps(obj, indent="\t"))
with open(f"./testfiles/result_{year}_{month}.json", "w", encoding="utf-8") as json_file:
    json_file.write(json.dumps(articleWordCount, indent="\t"))

In [6]:
# wtff
import multiprocessing
from tqdm.notebook import tqdm
import ctypes
import numpy as np
from utils import init, my_func

#-- edited 2015-05-01: the assert check below checks the wrong thing
#   with recent versions of Numpy/multiprocessing. That no copy is made
#   is indicated by the fact that the program prints the output shown below.
## No copy was made
##assert shared_array.base.base is shared_array_base.get_obj()
l = []
for i in range(10):
    l.append({"a": i, "b": {"c": i**2}})
    
shared_array_base = multiprocessing.Array(l)
docs = range(10)
with multiprocessing.Pool(processes=3, initializer=init, initargs=(shared_array_base,)) as p:
    with tqdm(total=len(docs), desc="Going through articles") as pbar:
        for res in p.imap_unordered(my_func, docs):
            pbar.update()

shared_array = np.ctypeslib.as_array(shared_array_base.get_obj())
shared_array = shared_array.reshape(10, 10)
print(shared_array)

None
None
None
None
None
None
None
None
None
None
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [3. 3. 3. 3. 3. 3. 3. 3. 3. 3.]
 [4. 4. 4. 4. 4. 4. 4. 4. 4. 4.]
 [5. 5. 5. 5. 5. 5. 5. 5. 5. 5.]
 [6. 6. 6. 6. 6. 6. 6. 6. 6. 6.]
 [7. 7. 7. 7. 7. 7. 7. 7. 7. 7.]
 [8. 8. 8. 8. 8. 8. 8. 8. 8. 8.]
 [9. 9. 9. 9. 9. 9. 9. 9. 9. 9.]]


In [4]:
# Test shared mem
from multiprocessing import Pool, Manager
from multiprocessing.shared_memory import SharedMemory
from tqdm.notebook import tqdm
from utils import *

l = []
for i in range(10):
    l.append({"a": i, "b": {"c": i**2}})

with Manager() as smm:
    sl = smm.list()
    sl.extend(l)
    print(setArr(l))
    print(getArr())
    with Pool(processes=3) as p:
        with tqdm(total=len(sl), desc="Going through articles") as pbar:
            for res in p.imap_unordered(mult, sl, chunksize=1):
                print(res)
                pbar.update()

[{'a': 0, 'b': {'c': 0}}, {'a': 1, 'b': {'c': 1}}, {'a': 2, 'b': {'c': 4}}, {'a': 3, 'b': {'c': 9}}, {'a': 4, 'b': {'c': 16}}, {'a': 5, 'b': {'c': 25}}, {'a': 6, 'b': {'c': 36}}, {'a': 7, 'b': {'c': 49}}, {'a': 8, 'b': {'c': 64}}, {'a': 9, 'b': {'c': 81}}]
[{'a': 0, 'b': {'c': 0}}, {'a': 1, 'b': {'c': 1}}, {'a': 2, 'b': {'c': 4}}, {'a': 3, 'b': {'c': 9}}, {'a': 4, 'b': {'c': 16}}, {'a': 5, 'b': {'c': 25}}, {'a': 6, 'b': {'c': 36}}, {'a': 7, 'b': {'c': 49}}, {'a': 8, 'b': {'c': 64}}, {'a': 9, 'b': {'c': 81}}]


Going through articles:   0%|          | 0/10 [00:00<?, ?it/s]

why? {'a': 2, 'b': {'c': 4}}, []
why? {'a': 1, 'b': {'c': 1}}, []
why? {'a': 0, 'b': {'c': 0}}, []
why? {'a': 3, 'b': {'c': 9}}, []
why? {'a': 4, 'b': {'c': 16}}, []
why? {'a': 5, 'b': {'c': 25}}, []
why? {'a': 6, 'b': {'c': 36}}, []
why? {'a': 7, 'b': {'c': 49}}, []
why? {'a': 8, 'b': {'c': 64}}, []
why? {'a': 9, 'b': {'c': 81}}, []


In [4]:
#There to compare different results of words
def getWords(article):
    return list(article["wordCount"].keys())
def flat(ll):
    ret = []
    for l in ll:
        ret.extend(l)
    return list(set(ret))

words = flat(list(map(lambda a: getWords(a), articleWordCount)))
words = sorted(words)
with open("./testfiles/result2.json", "w", encoding="utf-8") as json_file:
    json_file.write(json.dumps(words, indent="\t"))

In [7]:
import spacy
taggers = [{
    "name": "Taggers",
    "values": [
        "$", "''", ",", "-LRB-", "-RRB-", ".", ":", "ADD", "AFX", "CC", "CD", "DT", "EX", "FW", "HYPH", "IN", "JJ", "JJR", "JJS", "LS", "MD", "NFP", "NN", "NNP", "NNPS", "NNS", "PDT", "POS", "PRP", "PRP$", "RB", "RBR", "RBS", "RP", "SYM", "TO", "UH", "VB", "VBD", "VBG", "VBN", "VBP", "VBZ", "WDT", "WP", "WP$", "WRB", "XX", "_SP", "``"
    ]
}]

for tagger in taggers:
    print(f"---------{tagger['name']}----------")
    for tag in tagger["values"]:
        print(f"{tag}: {spacy.explain(tag)}")

---------Taggers----------
$: symbol, currency
'': closing quotation mark
,: punctuation mark, comma
-LRB-: left round bracket
-RRB-: right round bracket
.: punctuation mark, sentence closer
:: punctuation mark, colon or ellipsis
ADD: email
AFX: affix
CC: conjunction, coordinating
CD: cardinal number
DT: determiner
EX: existential there
FW: foreign word
HYPH: punctuation mark, hyphen
IN: conjunction, subordinating or preposition
JJ: adjective (English), other noun-modifier (Chinese)
JJR: adjective, comparative
JJS: adjective, superlative
LS: list item marker
MD: verb, modal auxiliary
NFP: superfluous punctuation
NN: noun, singular or mass
NNP: noun, proper singular
NNPS: noun, proper plural
NNS: noun, plural
PDT: predeterminer
POS: possessive ending
PRP: pronoun, personal
PRP$: pronoun, possessive
RB: adverb
RBR: adverb, comparative
RBS: adverb, superlative
RP: adverb, particle
SYM: symbol
TO: infinitival "to"
UH: interjection
VB: verb, base form
VBD: verb, past tense
VBG: verb, gerund

In [ ]:
import utils

t = "I ate food grasiously. He has eaten. She eats. They eat\nThere was terror. Terrorism is bad. There were terrorists."
lemmas = utils.get(text)
print(lemmas)
for l in lemmas:
    print(f"{l.lemma_}, {l.pos_}, {l.is_stop}, {l.is_punct}")

German Chancellor Angela Merkel died in 1936 in New York. She got shot by a mysterious terrorist, terror, terrorism
German, PROPN, False, False
Chancellor, PROPN, False, False
Angela Merkel, PROPN, False, False
die, VERB, False, False
in, ADP, True, False
1936, NUM, False, False
in, ADP, True, False
New York, PROPN, False, False
., PUNCT, False, True
she, PRON, True, False
get, VERB, False, False
shoot, VERB, False, False
by, ADP, True, False
a, DET, True, False
mysterious, ADJ, False, False
terrorist, NOUN, False, False
,, PUNCT, False, True
terror, NOUN, False, False
,, PUNCT, False, True
terrorism, NOUN, False, False


In [19]:
import os
import requests
import json
from multiprocessing import Pool, Manager
from tqdm.notebook import tqdm
from utils import parseArticle

## Initialize

apiKey = "hTLGygty2Sj5IB368rcArA63Xu29hW2r"
archiveUrl = f"https://api.nytimes.com/svc/archive/v1/#1/#2.json?api-key={apiKey}"


def getDocs(year, month):
    requestUrl = archiveUrl.replace("#1", f"{year}").replace("#2", f"{month}")
    print(requestUrl)
    res = requests.get(requestUrl)
    return json.loads(res.text)["response"]["docs"]


## Search

# https://stackoverflow.com/questions/5549190/is-shared-readonly-data-copied-to-different-processes-for-multiprocessing/5550156#5550156
# https://docs.python.org/3/library/multiprocessing.shared_memory.html
# python feed pool with shared memory
# https://stackoverflow.com/questions/77502344/how-do-shareablelists-actually-work-when-multiprocessing-in-python
def getWordCount(docs):
    articleWordCount = []
    # Brudi das zerstört meinen RAM komplett
    with Manager() as smm:
        sl = smm.list()
        sl.extend(docs)
        with Pool(processes=2) as p:
            with tqdm(total=len(sl), desc="Going through articles") as articlePbar:
                for res in p.imap_unordered(parseArticle, sl, chunksize=10):
                    articleWordCount.append(res)
                    articlePbar.update()
                # Use this to test topic fill
                # for res in p.imap_unordered(utils.test, docs):
                #    print(res)
                #    pbar.update()
    return articleWordCount


## Save results for testing

# with open("./testfiles/articles.json", "w", encoding="utf-8") as json_file:
#    json_file.write(json.dumps(obj, indent="\t"))
def saveWordCount(articleWordCount, year, month):
    with open(f"./resultfiles/result_{year}_{month}.json", "w", encoding="utf-8") as json_file:
        json_file.write(json.dumps(articleWordCount, indent="\t"))


## Fetch

year_min = 1964
year_max = 2000
month_min = 1
month_max = 13
with tqdm(total=((year_max - year_min) * 12), desc="Going through months", maxinterval=1) as pbar:
    for month in range(1, month_min):
        os.system('cls')
        pbar.display()
        print(f"current: {year_min}/{month}")
        docs = getDocs(year, month)
        wordCount = getWordCount(docs)
        saveWordCount(wordCount, year, month)
        pbar.update()

    for year in range(year_min, year_max):
        for month in range(month_min, month_max):
            os.system('cls')
            pbar.display()
            print(f"current: {year}/{month}")
            docs = getDocs(year, month)
            wordCount = getWordCount(docs)
            saveWordCount(wordCount, year, month)
            pbar.update()


Going through months:   0%|          | 0/432 [00:00<?, ?it/s]

current: 1964/1
https://api.nytimes.com/svc/archive/v1/1964/1.json?api-key=hTLGygty2Sj5IB368rcArA63Xu29hW2r


Going through articles:   0%|          | 0/12273 [00:00<?, ?it/s]

current: 1964/2
https://api.nytimes.com/svc/archive/v1/1964/2.json?api-key=hTLGygty2Sj5IB368rcArA63Xu29hW2r


Going through articles:   0%|          | 0/10739 [00:00<?, ?it/s]

current: 1964/3
https://api.nytimes.com/svc/archive/v1/1964/3.json?api-key=hTLGygty2Sj5IB368rcArA63Xu29hW2r


Going through articles:   0%|          | 0/11877 [00:00<?, ?it/s]

current: 1964/4
https://api.nytimes.com/svc/archive/v1/1964/4.json?api-key=hTLGygty2Sj5IB368rcArA63Xu29hW2r


Going through articles:   0%|          | 0/11460 [00:00<?, ?it/s]

KeyboardInterrupt: 